# Supervised Learning Practise

### Importing Data Files

In [98]:
#Import Product DataSet here
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

body = client_864211817f7746c883c6ad3e846cdb85.get_object(Bucket='project1-donotdelete-pr-hoinwpdndybmjj',Key='Product Data Set - Student 2 of 3.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

product_data = pd.read_csv(body,sep='|')
product_data.head()


,PRODUCT CODE,PRODUCT CATEGORY,UNIT LIST PRICE
0,30001,HEALTH & BEAUTY,$7.45
1,30002,HEALTH & BEAUTY,$5.35
2,30003,HEALTH & BEAUTY,$5.49
3,30004,HEALTH & BEAUTY,$6.46
4,30005,HEALTH & BEAUTY,$7.33


In [2]:
#Import Transaction DataSet Here
body = client_864211817f7746c883c6ad3e846cdb85.get_object(Bucket='project1-donotdelete-pr-hoinwpdndybmjj',Key='Transaction Data Set - Student 3 of 3.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

transactions_data = pd.read_csv(body,sep='|')
transactions_data.head()


,CUSTOMER NUM,PRODUCT NUM,QUANTITY PURCHASED,DISCOUNT TAKEN,TRANSACTION DATE,STOCKOUT
0,10114,30011,4,0.0,1/2/2015,0
1,10217,30016,3,0.0,1/2/2015,0
2,10224,30013,4,0.0,1/2/2015,0
3,10103,30012,3,0.2,1/2/2015,0
4,10037,30010,8,0.0,1/2/2015,0


In [3]:
#Import Customer Dataset Here
body = client_864211817f7746c883c6ad3e846cdb85.get_object(Bucket='project1-donotdelete-pr-hoinwpdndybmjj',Key='Customer Data Set - Student 1 of 3.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

customer_data=pd.read_csv(body)
customer_data.head()


,CUSTOMERID,GENDER,AGE,INCOME,EXPERIENCE SCORE,LOYALTY GROUP,ENROLLMENT DATE,HOUSEHOLD SIZE,MARITAL STATUS
0,10001,0,64,"$133,498",5,enrolled,06-03-2013,4,Single
1,10002,0,42,"$94,475",9,notenrolled,NaN,6,Married
2,10003,0,40,"$88,610",9,enrolled,02-09-2010,5,Married
3,10004,0,38,"$84,313",8,enrolled,06-04-2015,1,Single
4,10005,0,30,"$51,498",3,notenrolled,NaN,1,Single


### Changing Data Types

In [4]:
customer_data['INCOME']=customer_data['INCOME'].map(lambda x : x.replace('$',''))

In [5]:
customer_data['INCOME']=customer_data['INCOME'].map(lambda x : int(x.replace(',','')))

### Creating Customer View

In [6]:
trans_products=transactions_data.merge(product_data,how='inner', left_on='PRODUCT NUM', right_on='PRODUCT CODE')

In [7]:
trans_products['UNIT LIST PRICE']=trans_products['UNIT LIST PRICE'].map(lambda x : float(x.replace('$','')))

In [8]:
trans_products['Total_Price']=trans_products['QUANTITY PURCHASED'] * trans_products['UNIT LIST PRICE'] * (1- trans_products['DISCOUNT TAKEN'])

In [10]:
customer_prod_categ=trans_products.groupby(['CUSTOMER NUM','PRODUCT CATEGORY']).agg({'Total_Price':'sum'})
customer_prod_categ.head()

Total_Price
CUSTOMER NUM PRODUCT CATEGORY             
10001        APPAREL              4022.430
             ELECTRONICS          1601.315
             FOOD                   68.688
             HEALTH & BEAUTY      1134.337
10002        APPAREL              2312.509

In [12]:
customer_prod_categ=customer_prod_categ.reset_index()
customer_prod_categ.head()

,index,CUSTOMER NUM,PRODUCT CATEGORY,Total_Price
0,0,10001,APPAREL,4022.430
1,1,10001,ELECTRONICS,1601.315
2,2,10001,FOOD,68.688
3,3,10001,HEALTH & BEAUTY,1134.337
4,4,10002,APPAREL,2312.509


In [13]:
customer_pivot=customer_prod_categ.pivot(index='CUSTOMER NUM',columns='PRODUCT CATEGORY',values='Total_Price')
customer_pivot.head()

PRODUCT CATEGORY,APPAREL,ELECTRONICS,FOOD,HEALTH & BEAUTY
CUSTOMER NUM,,,,
10001,4022.430,1601.315,68.688,1134.337
10002,2312.509,2473.163,276.779,NaN
10003,2887.382,5414.418,260.640,NaN
10004,3637.213,1840.211,45.270,NaN
10005,213.512,NaN,NaN,NaN


In [14]:
trans_total_spend=trans_products.groupby('CUSTOMER NUM').agg({'Total_Price':'sum'}).\
rename(columns={'Total_Price':'TOTAL SPENT'})

In [15]:
customer_KPIs=customer_pivot.merge(trans_total_spend,how='inner',left_index=True, right_index=True )

In [16]:
customer_KPIs=customer_KPIs.fillna(0)


In [17]:
customer_all_view=customer_data.merge(customer_KPIs,how='inner', left_on='CUSTOMERID', right_index=True)

In [18]:
customer_all_view.head()

,CUSTOMERID,GENDER,AGE,INCOME,EXPERIENCE SCORE,LOYALTY GROUP,ENROLLMENT DATE,HOUSEHOLD SIZE,MARITAL STATUS,APPAREL,ELECTRONICS,FOOD,HEALTH & BEAUTY,TOTAL SPENT
0,10001,0,64,133498,5,enrolled,06-03-2013,4,Single,4022.430,1601.315,68.688,1134.337,6826.770
1,10002,0,42,94475,9,notenrolled,NaN,6,Married,2312.509,2473.163,276.779,0.000,5062.451
2,10003,0,40,88610,9,enrolled,02-09-2010,5,Married,2887.382,5414.418,260.640,0.000,8562.440
3,10004,0,38,84313,8,enrolled,06-04-2015,1,Single,3637.213,1840.211,45.270,0.000,5522.694
4,10005,0,30,51498,3,notenrolled,NaN,1,Single,213.512,0.000,0.000,0.000,213.512


# Linear Regression 

In [ ]:
#Begin Writing your code here

In [19]:
DF_input=customer_all_view[['GENDER','AGE','INCOME','EXPERIENCE SCORE','LOYALTY GROUP','HOUSEHOLD SIZE','MARITAL STATUS']]

In [20]:
DF_input.head()

,GENDER,AGE,INCOME,EXPERIENCE SCORE,LOYALTY GROUP,HOUSEHOLD SIZE,MARITAL STATUS
0,0,64,133498,5,enrolled,4,Single
1,0,42,94475,9,notenrolled,6,Married
2,0,40,88610,9,enrolled,5,Married
3,0,38,84313,8,enrolled,1,Single
4,0,30,51498,3,notenrolled,1,Single


In [21]:
DF_input["MARITAL STATUS"].unique()

array(['Single', 'Married', 'Divorced', 'Widow/Widower'], dtype=object)

In [22]:
def encode_loyalty(value):
    if value == "enrolled" :
        return 1 
    else : 
        return 0

In [23]:
DF_input['LOYALTY GROUP']=DF_input['LOYALTY GROUP'].apply(encode_loyalty)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [24]:
DF_input.head()

,GENDER,AGE,INCOME,EXPERIENCE SCORE,LOYALTY GROUP,HOUSEHOLD SIZE,MARITAL STATUS
0,0,64,133498,5,1,4,Single
1,0,42,94475,9,0,6,Married
2,0,40,88610,9,1,5,Married
3,0,38,84313,8,1,1,Single
4,0,30,51498,3,0,1,Single


In [25]:
DF_input=pd.get_dummies(DF_input)

In [26]:
DF_input.head()

,GENDER,AGE,INCOME,EXPERIENCE SCORE,LOYALTY GROUP,HOUSEHOLD SIZE,MARITAL STATUS_Divorced,MARITAL STATUS_Married,MARITAL STATUS_Single,MARITAL STATUS_Widow/Widower
0,0,64,133498,5,1,4,0,0,1,0
1,0,42,94475,9,0,6,0,1,0,0
2,0,40,88610,9,1,5,0,1,0,0
3,0,38,84313,8,1,1,0,0,1,0
4,0,30,51498,3,0,1,0,0,1,0


In [28]:
from sklearn import preprocessing
DF_input_column_names = DF_input.columns.values
DF_input_np=preprocessing.minmax_scale(DF_input)
Reg_input_scaled=pd.DataFrame(DF_input_np, columns=DF_input_column_names)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64.
  app.launch_new_instance()


In [29]:
Reg_input_scaled.head()

,GENDER,AGE,INCOME,EXPERIENCE SCORE,LOYALTY GROUP,HOUSEHOLD SIZE,MARITAL STATUS_Divorced,MARITAL STATUS_Married,MARITAL STATUS_Single,MARITAL STATUS_Widow/Widower
0,0.0,0.638889,0.872818,0.444444,1.0,0.6,0.0,0.0,1.0,0.0
1,0.0,0.333333,0.572046,0.888889,0.0,1.0,0.0,1.0,0.0,0.0
2,0.0,0.305556,0.526842,0.888889,1.0,0.8,0.0,1.0,0.0,0.0
3,0.0,0.277778,0.493722,0.777778,1.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.166667,0.240799,0.222222,0.0,0.0,0.0,0.0,1.0,0.0


In [30]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(Reg_input_scaled,customer_all_view['TOTAL SPENT'], test_size=0.2,random_state=42)

In [31]:
from sklearn.linear_model import LinearRegression

In [32]:
regr = LinearRegression()

In [33]:
regr.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [34]:
Y_pred = regr.predict(X_test)

In [35]:
print('Coefficients: \n', regr.coef_)

Coefficients: 
 [   35.58494877 -7076.18001534 12403.0952701    505.84578936
  1566.48092344  1040.19923196   377.33573376   -82.02665324
   -34.57620088  -260.73287964]


In [36]:
Reg_input_scaled.columns.values

array(['GENDER', 'AGE', 'INCOME', 'EXPERIENCE SCORE', 'LOYALTY GROUP',
       'HOUSEHOLD SIZE', 'MARITAL STATUS_Divorced',
       'MARITAL STATUS_Married', 'MARITAL STATUS_Single',
       'MARITAL STATUS_Widow/Widower'], dtype=object)

In [71]:
predictors = {}
count = 0
for i in Reg_input_scaled.columns.values:
    predictors[i] = regr.coef_[count]
    count += 1

In [72]:
predictors

{'GENDER': 35.584948770712344,
 'AGE': -7076.180015341927,
 'INCOME': 12403.095270099202,
 'EXPERIENCE SCORE': 505.8457893621816,
 'LOYALTY GROUP': 1566.480923435115,
 'HOUSEHOLD SIZE': 1040.1992319563028,
 'MARITAL STATUS_Divorced': 377.33573376215315,
 'MARITAL STATUS_Married': -82.02665324017252,
 'MARITAL STATUS_Single': -34.57620088163161,
 'MARITAL STATUS_Widow/Widower': -260.73287964031255}

In [93]:
predict = pd.DataFrame.from_dict(predictors, orient='index', columns=['Coefficients'])

In [94]:
def rela(series):
    if series > 0:
        series = 'Positive'
    else:
        series='Negative'
    return series

predict['Relationship with Total Spend'] = predict['Coefficients'].apply(rela)

In [95]:
predict.sort_values(by='Coefficients')

,Coefficients,Relationship with Total Spend
AGE,-7076.180015,Negative
MARITAL STATUS_Widow/Widower,-260.732880,Negative
MARITAL STATUS_Married,-82.026653,Negative
MARITAL STATUS_Single,-34.576201,Negative
GENDER,35.584949,Positive
MARITAL STATUS_Divorced,377.335734,Positive
EXPERIENCE SCORE,505.845789,Positive
HOUSEHOLD SIZE,1040.199232,Positive
LOYALTY GROUP,1566.480923,Positive
INCOME,12403.095270,Positive


In [96]:
print('intercept: \n', regr.intercept_)


intercept: 
 -38.16947521866132


Predicted Total Spend = 35.58 * Gender - 7076 * AGE + 12403 * Income + 505 * Experience score + 155 * Loyalty group + 1040 * House hold size + 377.3 * (Marital Status – Divorced) – 82.02 * (Marital Status – Married) - 35.57 * (Marital Status – Single) - 260 * (Marital Status – Widow) - 38.17

In [97]:
from sklearn.metrics import mean_squared_error
print("Mean squared error: %.2f" % mean_squared_error(Y_test, Y_pred))

Mean squared error: 3164098.64


This is the mean squared of the error. To get the mean error, take the square root of this number. So, the mean error is 1779. Therefore, our linear model predicts total spending with an average error of $1779